In [1]:
pwd()

"/Users/arnab/arnab/workspace/SPF-data/cr-qubit"

In [2]:
using Pkg; Pkg.activate("../../FermiCG/")
using FermiCG, NPZ,JLD2
readdir()


  Activating project at `~/arnab/workspace/FermiCG`


30-element Vector{String}:
 "C_act_loc_sorted_3.molden"
 "README.md"
 "b3lyp-d3_def2svp.xyz"
 "cluster_bases_2.jld2"
 "clustered1_integrals_h0.npy"
 "clustered1_integrals_h1.npy"
 "clustered1_integrals_h2.npy"
 "clustered1_mo_coeffs.npy"
 "clustered1_overlap_mat.npy"
 "cmf_1.jl"
 ⋮
 "cr_qubit_scf_mo_coeffs.npz.npy"
 "excitations1.xlsx"
 "scf.ipynb"
 "spt_1.jl"
 "spt_2.jl"
 "uhf_density.npy"
 "uhf_mo_coeffs_a.npy"
 "uhf_mo_coeffs_b.npy"
 "wb97x-v_def2svp.xyz"

In [3]:
using JLD2
@load "cmf_data_2.jld2"

#
# form Cluster data
# cluster_bases = FermiCG.compute_cluster_eigenbasis_spin(ints, clusters, d1, [3,3,3,3,1,4], FockConfig(init_fspace), max_roots=20, verbose=1);
@load "cluster_bases_2.jld2"

clustered_ham = FermiCG.extract_ClusteredTerms(ints, clusters)

cluster_ops = FermiCG.compute_cluster_ops(cluster_bases, ints);

FermiCG.add_cmf_operators!(cluster_ops, cluster_bases, ints, d1.a, d1.b);





 Remove duplicates
 Number of terms reduced from  5226 to  2226
IDX001:DIM4096:001|002|003|004|005|006|
IDX002:DIM4096:007|008|009|010|011|012|
IDX003:DIM4096:013|014|015|016|017|018|
IDX004:DIM4096:019|020|021|022|023|024|
IDX005:DIM0016:025|026|
IDX006:DIM65536:027|028|029|030|031|032|033|034|


In [5]:
# Create BST state

v = FermiCG.BSTstate(clusters, FockConfig(init_fspace), cluster_bases, R=3)

FermiCG.add_single_excitons!(v, FockConfig(init_fspace),3)
FermiCG.randomize!(v)
FermiCG.orthonormalize!(v)
e_ci, v = FermiCG.ci_solve(v, cluster_ops, clustered_ham);


 Tucker Initialize:(19, 1, 1, 1, 1, 1)
 index dimension:     19
 index dimension:      1
 index dimension:      1
 index dimension:      1
 index dimension:      1
 index dimension:      1
 Tucker Initialize:(1, 19, 1, 1, 1, 1)
 index dimension:      1
 index dimension:     19
 index dimension:      1
 index dimension:      1
 index dimension:      1
 index dimension:      1
 Tucker Initialize:(1, 1, 19, 1, 1, 1)
 index dimension:      1
 index dimension:      1
 index dimension:     19
 index dimension:      1
 index dimension:      1
 index dimension:      1
 Tucker Initialize:(1, 1, 1, 19, 1, 1)
 index dimension:      1
 index dimension:      1
 index dimension:      1
 index dimension:     19
 index dimension:      1
 index dimension:      1
 Tucker Initialize:(1, 1, 1, 1, 3, 1)
 index dimension:      1
 index dimension:      1
 index dimension:      1
 index dimension:      1
 index dimension:      3
 index dimension:      1
 Tucker Initialize:(1, 1, 1, 1, 1, 19)
 index dimension:

In [6]:
using Printf
display(v, root=2)
for ei in 1:length(e_ci)
    @printf(" %5i %12.8f %12.8f eV\n", ei, e_ci[ei]+ints.h0, (e_ci[ei]-e_ci[1])*27.21165)
end


 --------------------------------------------------
 ---------- # Fockspaces -------------------:     1  
 ---------- # Configs    -------------------:    19  
 ---------- Root ---------------------------:     2  
 --------------------------------------------------
 Printing contributions greater than: 0.010000
 Weight              # configs (full)    (α,β)...            
 -------             --------- --------- ----------          
 1.000                                   ( 3,3 )( 3,3 )( 3,3 )( 3,3 )( 1,1 )( 4,4 )
     1.000           3         19            1:1    1:1    1:1    1:1    1:1   2:20
 ---------                               ----------          
 --------------------------------------------------

     1 -2119.15817765   0.00000000 eV
     2 -2119.08108600   2.09779088 eV
     3 -2119.08072816   2.10752834 eV


In [9]:
 e_var, v_var = FermiCG.block_sparse_tucker(v, cluster_ops, clustered_ham,
                                               max_iter    = 40,
                                               nbody       = 4,
                                               H0          = "Hcmf",
                                               thresh_var  = 1e-1,
                                               thresh_foi  = 1e-3,
                                               thresh_pt   = 1e-3,
                                               ci_conv     = 1e-5,
                                               do_pt       = true,
                                               resolve_ss  = false,
                                               tol_tucker  = 1e-4,
                                               solver      = "davidson")



 max_iter         : 40
 nbody            : 4
 H0               : Hcmf
 thresh_var       : 0.1
 thresh_foi       : 0.001
 thresh_pt        : 0.001
 thresh_spin      : nothing
 ci_conv          : 1.0e-5
 ci_max_iter      : 50
 ci_max_ss_vecs   : 12
 ci_lindep_thresh : 1.0e-10
 resolve_ss       : false
 do_pt            : true
 tol_tucker       : 0.0001



     BST Iteration:    1 epsilon:   0.10000000
 Ref state compressed from:                                19 → 3          (thresh_var  =  1.0e-01)
 Compute zeroth-order energy:                        0.002922 seconds (34.48 k allocations: 2.668 MiB)
 Compute <S^2>:                                      0.005804 seconds (10.30 k allocations: 791.406 KiB, 67.58% compilation time)
  Root       Energy           S2
     1 -105.89833458   2.00000000
     2 -105.82122988   0.00000000
     3 -105.82088414   0.00000000

 In build_compressed_1st_order_state
 Setup threaded jobs:                                0.000391 seconds (4.01 k allocations: 

([-105.91966205987248, -105.83795189662477, -105.83730656070756], BSTstate{Float64, 6, 3}(MOCluster[MOCluster(1, Int16[1, 2, 3, 4, 5, 6]), MOCluster(2, Int16[7, 8, 9, 10, 11, 12]), MOCluster(3, Int16[13, 14, 15, 16, 17, 18]), MOCluster(4, Int16[19, 20, 21, 22, 23, 24]), MOCluster(5, Int16[25, 26]), MOCluster(6, Int16[27, 28, 29, 30, 31, 32, 33, 34])], OrderedCollections.OrderedDict{FockConfig{6}, OrderedCollections.OrderedDict{TuckerConfig{6}, FermiCG.Tucker{Float64, 6, 3}}}(FockConfig{6}(((3, 3), (3, 3), (3, 3), (3, 3), (1, 1), (4, 4))) => OrderedCollections.OrderedDict(TuckerConfig{6}((1:1, 1:1, 1:1, 1:1, 1:1, 1:1)) => FermiCG.Tucker{Float64, 6, 3}(([0.9824990784130103;;;;;;], [2.6581601109516452e-12;;;;;;], [9.028108104635331e-11;;;;;;]), ([1.0;;], [1.0;;], [1.0;;], [1.0;;], [1.0;;], [1.0;;])), TuckerConfig{6}((1:1, 1:1, 1:1, 1:1, 1:1, 2:20)) => FermiCG.Tucker{Float64, 6, 3}(([-1.1387366122488009e-11;;;;;; -8.635594852597215e-11], [0.976328976753062;;;;;; -0.009225840686989164], [0.

In [10]:
@time e2 = FermiCG.compute_pt2_energy(v_var, cluster_ops, clustered_ham, thresh_foi=1e-5)
# @time e2 = FermiCG.compute_pt2_energy(v_var, cluster_ops, clustered_ham, thresh_foi=1e-6)
# @time e2 = FermiCG.compute_pt2_energy(v_var, cluster_ops, clustered_ham, thresh_foi=1e-7)


 |...................................BST-PT2............................................
 H0          : Hcmf
 nbody       : 4
 thresh_foi  : 1.0e-5
 max_number  : nothing
 opt_ref     : true
 ci_tol      : 1.0e-6
 verbose     : 1

 Length of Reference:                                    1542
 Solve zeroth-order problem:                       
 |== BST CI ========================================================
 Solve CI with # variables:                              1542
 Cache Hamiltonian:                                  3.189610 seconds (13.58 M allocations: 1.123 GiB, 4.36% gc time)
 Iter:   1 SS: 3    E: -105.91966206* -105.83795190  -105.83730656   R: 3.3e-07* 7.4e-06  5.4e-06   LinDep: 2.2e-16* 
 Iter:   2 SS: 6    E: -105.91966206* -105.83795190  -105.83730656   R: 1.7e-07* 3.5e-06  2.6e-06   LinDep: 1.3e-15* 
 Iter:   3 SS: 9    E: -105.91966206* -105.83795190  -105.83730656   R: 8.1e-08* 1.9e-06  1.4e-06   LinDep: 1.7e-15* 
 Iter:   4 SS: 12   E: -105.91966206* -105.83795190

CompositeException: TaskFailedException

    nested task error: TaskFailedException
    
        nested task error: InterruptException:
        Stacktrace:
          [1] Array
            @ ./boot.jl:489 [inlined]
          [2] similar
            @ ./array.jl:375 [inlined]
          [3] permutedims(B::Array{Float64, 6}, perm::Vector{Int64})
            @ Base ./multidimensional.jl:1549
          [4] contract_dense_H_with_state(term::FermiCG.ClusteredTerm3B{Float64}, op::Array{Float64, 6}, state_sign::Int64, coeffs_bra::FermiCG.Tucker{Float64, 6, 3}, coeffs_ket::FermiCG.Tucker{Float64, 6, 3})
            @ FermiCG ~/arnab/workspace/FermiCG/src/tucker_contract_dense_H_with_state.jl:239
          [5] form_sigma_block!(term::FermiCG.ClusteredTerm3B{Float64}, cluster_ops::Vector{FermiCG.ClusterOps{Float64}}, fock_bra::FockConfig{6}, bra::TuckerConfig{6}, fock_ket::FockConfig{6}, ket::TuckerConfig{6}, coeffs_bra::FermiCG.Tucker{Float64, 6, 3}, coeffs_ket::FermiCG.Tucker{Float64, 6, 3}, scr_

In [11]:
v1 = deepcopy(v_var);
display(v1,root=3)


 --------------------------------------------------
 ---------- # Fockspaces -------------------:   155  
 ---------- # Configs    -------------------:  1542  
 ---------- Root ---------------------------:     3  
 --------------------------------------------------
 Printing contributions greater than: 0.010000
 Weight              # configs (full)    (α,β)...            
 -------             --------- --------- ----------          
 0.978                                   ( 3,3 )( 3,3 )( 3,3 )( 3,3 )( 1,1 )( 4,4 )
     0.977           2         19            1:1    1:1    1:1    1:1    1:1   2:20
     0.011           8         6859         2:20    1:1   2:20    1:1    1:1   2:20
     0.011           8         6859          1:1   2:20   2:20    1:1    1:1   2:20
     0.011           8         6859         2:20    1:1    1:1   2:20    1:1   2:20
     0.011           8         6859          1:1   2:20    1:1   2:20    1:1   2:20
     0.011           8         6859          1:1    1:1   

In [12]:
 e2, v2 = FermiCG.block_sparse_tucker(v1, cluster_ops, clustered_ham,
                                        max_iter    = 20,
                                        nbody       = 4,
                                        H0          = "Hcmf",
                                        thresh_var  = 5e-2,
                                        thresh_foi  = 1e-4,
                                        thresh_pt   = 1e-3,
                                        ci_conv     = 1e-5,
                                        do_pt       = true,                          
                                        resolve_ss  = false,
                                        tol_tucker  = 1e-4);

 max_iter         : 20
 nbody            : 4
 H0               : Hcmf
 thresh_var       : 0.05
 thresh_foi       : 0.0001
 thresh_pt        : 0.001
 thresh_spin      : nothing
 ci_conv          : 1.0e-5
 ci_max_iter      : 50
 ci_max_ss_vecs   : 12
 ci_lindep_thresh : 1.0e-10
 resolve_ss       : false
 do_pt            : true
 tol_tucker       : 0.0001



     BST Iteration:    1 epsilon:   0.05000000
 Ref state compressed from:                              1542 → 19         (thresh_var  =  5.0e-02)
 Compute zeroth-order energy:                        0.068379 seconds (342.44 k allocations: 28.817 MiB, 24.85% gc time)
 Compute <S^2>:                                      0.005454 seconds (46.40 k allocations: 3.397 MiB)
  Root       Energy           S2
     1 -105.90111920   2.00000454
     2 -105.82770234   0.00499157
     3 -105.82705048   0.00232877

 In build_compressed_1st_order_state
 Setup threaded jobs:                                0.003323 seconds (28.08 k allocations: 2.484 

┌ Warning: ("initial guess det(v0'v0) = ", 1.0000000000000127)
└ @ FermiCG /Users/arnab/arnab/workspace/FermiCG/src/type_BSTstate.jl:460


  4.239477 seconds (17.62 M allocations: 1.486 GiB, 11.12% gc time)
 Iter:   1 SS: 3    E: -105.92023370  -105.83917826  -105.83845408   R: 4.6e-04  1.5e-03  2.3e-03   LinDep: 1.0e-15* 
 Iter:   2 SS: 6    E: -105.92023405  -105.83918184  -105.83846240   R: 1.2e-04  2.3e-04  2.7e-04   LinDep: 2.2e-16* 
 Iter:   3 SS: 9    E: -105.92023407  -105.83918198  -105.83846263   R: 7.5e-05  1.9e-04  1.5e-04   LinDep: 0.0e+00* 
 Iter:   4 SS: 12   E: -105.92023408  -105.83918204  -105.83846268   R: 3.1e-05  1.2e-04  1.1e-04   LinDep: 7.8e-16* 
 Iter:   5 SS: 15   E: -105.92023408  -105.83918208  -105.83846269   R: 1.6e-05  6.5e-05  5.0e-05   LinDep: 6.7e-16* 
 Iter:   6 SS: 18   E: -105.92023409* -105.83918209  -105.83846270   R: 9.6e-06* 4.0e-05  3.4e-05   LinDep: 1.1e-15* 
 Iter:   7 SS: 21   E: -105.92023409* -105.83918209  -105.83846270   R: 4.3e-06* 2.5e-05  2.4e-05   LinDep: 2.9e-15* 
 Iter:   8 SS: 24   E: -105.92023409* -105.83918209  -105.83846270   R: 1.9e-06* 1.3e-05  1.3e-05   LinDep

In [13]:
display(v2, root=1, thresh=.1)



 --------------------------------------------------
 ---------- # Fockspaces -------------------:   207  
 ---------- # Configs    -------------------:  2824  
 ---------- Root ---------------------------:     1  
 --------------------------------------------------
 Printing contributions greater than: 0.100000
 Weight              # configs (full)    (α,β)...            
 -------             --------- --------- ----------          
 0.983                                   ( 3,3 )( 3,3 )( 3,3 )( 3,3 )( 1,1 )( 4,4 )
     0.981           1         1             1:1    1:1    1:1    1:1    1:1    1:1
 ---------                               ----------          
 --------------------------------------------------



In [14]:
 e3, v3 = FermiCG.block_sparse_tucker(v2, cluster_ops, clustered_ham,
                                        max_iter    = 20,
                                        nbody       = 4,
                                        H0          = "Hcmf",
                                        thresh_var  = 5e-2,
                                        thresh_foi  = 1e-5,
                                        thresh_pt   = 1e-4,
                                        ci_conv     = 1e-5,
                                        do_pt       = true,                          
                                        resolve_ss  = false,
                                        tol_tucker  = 1e-4);

 max_iter         : 20
 nbody            : 4
 H0               : Hcmf
 thresh_var       : 0.05
 thresh_foi       : 1.0e-5
 thresh_pt        : 0.0001
 thresh_spin      : nothing
 ci_conv          : 1.0e-5
 ci_max_iter      : 50
 ci_max_ss_vecs   : 12
 ci_lindep_thresh : 1.0e-10
 resolve_ss       : false
 do_pt            : true
 tol_tucker       : 0.0001



     BST Iteration:    1 epsilon:   0.05000000
 Ref state compressed from:                              2824 → 19         (thresh_var  =  5.0e-02)
 Compute zeroth-order energy:                        0.058298 seconds (342.43 k allocations: 28.817 MiB, 32.27% gc time)
 Compute <S^2>:                                      0.003073 seconds (46.40 k allocations: 3.382 MiB)
  Root       Energy           S2
     1 -105.90110638   2.00000218
     2 -105.82775692   0.00451022
     3 -105.82714146   0.00172918

 In build_compressed_1st_order_state
 Setup threaded jobs:                                0.002930 seconds (28.08 k allocations: 2.484

┌ Warning: ("initial guess det(v0'v0) = ", 1.0000000000000338)
└ @ FermiCG /Users/arnab/arnab/workspace/FermiCG/src/type_BSTstate.jl:460
┌ Warning: ("initial guess det(v0'v0) = ", 1.0000000000002758)
└ @ FermiCG /Users/arnab/arnab/workspace/FermiCG/src/type_BSTstate.jl:460
┌ Warning: ("initial guess det(v0'v0) = ", 1.0000000000002762)
└ @ FermiCG /Users/arnab/arnab/workspace/FermiCG/src/type_BSTstate.jl:460


 48.593628 seconds (145.56 M allocations: 16.329 GiB, 12.23% gc time)
 Iter:   1 SS: 3    E: -105.92025604  -105.83921285  -105.83849521   R: 2.4e-02  2.5e-02  2.6e-02   LinDep: 6.7e-16* 
 Iter:   2 SS: 6    E: -105.92081140  -105.83988197  -105.83916832   R: 6.7e-03  8.2e-03  7.9e-03   LinDep: 0.0e+00* 
 Iter:   3 SS: 9    E: -105.92086364  -105.83996616  -105.83924344   R: 2.9e-03  4.3e-03  3.8e-03   LinDep: 8.9e-16* 
 Iter:   4 SS: 12   E: -105.92087548  -105.83999774  -105.83926719   R: 1.5e-03  2.6e-03  2.2e-03   LinDep: 2.4e-15* 
 Iter:   5 SS: 15   E: -105.92087856  -105.84000927  -105.83927472   R: 8.4e-04  1.7e-03  1.4e-03   LinDep: 6.0e-15* 
 Iter:   6 SS: 18   E: -105.92087949  -105.84001365  -105.83927725   R: 5.7e-04  1.1e-03  7.8e-04   LinDep: 2.2e-14* 
 Iter:   7 SS: 21   E: -105.92087990  -105.84001546  -105.83927807   R: 4.0e-04  8.4e-04  5.3e-04   LinDep: 2.5e-14* 
 Iter:   8 SS: 24   E: -105.92088012  -105.84001653  -105.83927848   R: 2.6e-04  6.3e-04  3.9e-04   LinD

┌ Warning: ("initial guess det(v0'v0) = ", 1.0000000000000462)
└ @ FermiCG /Users/arnab/arnab/workspace/FermiCG/src/type_BSTstate.jl:460
┌ Warning: ("initial guess det(v0'v0) = ", 0.999999999999976)
└ @ FermiCG /Users/arnab/arnab/workspace/FermiCG/src/type_BSTstate.jl:460
┌ Warning: ("initial guess det(v0'v0) = ", 0.999999999999976)
└ @ FermiCG /Users/arnab/arnab/workspace/FermiCG/src/type_BSTstate.jl:460


 Cache Hamiltonian:                                 44.506335 seconds (144.08 M allocations: 16.103 GiB, 11.80% gc time)
 Iter:   1 SS: 3    E: -105.92088063  -105.84002240  -105.83928255   R: 6.8e-04  9.7e-04  6.4e-04   LinDep: 8.9e-16* 
 Iter:   2 SS: 6    E: -105.92088107  -105.84002337  -105.83928305   R: 1.6e-04  2.7e-04  2.3e-04   LinDep: 7.8e-15* 
 Iter:   3 SS: 9    E: -105.92088111  -105.84002349  -105.83928313   R: 5.7e-05  1.1e-04  8.7e-05   LinDep: 2.1e-14* 
 Iter:   4 SS: 12   E: -105.92088111  -105.84002351  -105.83928314   R: 2.8e-05  6.6e-05  5.2e-05   LinDep: 2.4e-14* 
 Iter:   5 SS: 15   E: -105.92088111  -105.84002352  -105.83928315   R: 1.4e-05  4.6e-05  3.3e-05   LinDep: 2.4e-14* 
 Iter:   6 SS: 18   E: -105.92088111* -105.84002352  -105.83928315   R: 7.3e-06* 3.1e-05  1.9e-05   LinDep: 1.6e-15* 
 Iter:   7 SS: 21   E: -105.92088111* -105.84002352  -105.83928315   R: 4.1e-06* 2.2e-05  1.3e-05   LinDep: 2.2e-15* 
 Iter:   8 SS: 24   E: -105.92088111* -105.84002353  